In [21]:
import json
import pandas as pd
from Levenshtein import distance
import geopandas as gpd
pd.set_option('display.max_rows', None)

In [22]:
tif_data = pd.read_csv("PhillipYates/master_TIF.csv", usecols=['tif_name', 'tif_year', 'transfers_in', 'transfers_out', 'property_tax_extraction'])

In [23]:
def extract_names_from_geojson(geojson_file):
    with open(geojson_file, 'r') as file:
        data = json.load(file)
        names = []
        if data['type'] == 'FeatureCollection':
            for feature in data['features']:
                if 'name' in feature['properties']:
                    names.append(feature['properties']['name'])
        elif data['type'] == 'Feature':
            if 'name' in data['properties']:
                names.append(data['properties']['name'])
        return names

def get_match(str1:str, values:list[str]):
    closest_distance = float('inf')
    closest_value = None
    for value2 in values:
        if value2 == "SBIF":
            continue
        if ((False not in [word.lower() in value2.lower() for word in str1.split(' ')]) or 
            (False not in [word.lower() in str1.lower() for word in value2.split(' ')])):
            return value2
        dist = distance(str1.lower(), value2.lower())
        if dist < closest_distance:
            closest_distance = dist
            closest_value = value2
    return closest_value

def find_closest_match(list1:list[str], list2:list[str], limbo=False):    
    limbo = []

    # Find closest match for each element in col1
    closest_mapping = {}
    closest_mapping_swapped = {None: []}

    for TIF in list1:
        closest_value = get_match(TIF, list2)
        
        if (limbo):
            if (closest_value in closest_mapping_swapped):
                limbo.append((TIF, []))
            
            while (limbo):
                new_key = limbo[0][0]
                tries = limbo[0][1]
                value = get_match(TIF, [x for x in list1 if x not in tries])
                set_key = closest_mapping_swapped[value] if value in closest_mapping_swapped else None

                if not value:
                    closest_mapping[new_key] = value
                    closest_mapping_swapped[value].append(new_key)
                    limbo.pop()
                elif (set_key == None):
                    closest_mapping[new_key] = value
                    closest_mapping_swapped[value] = new_key
                    limbo.pop()
                elif (distance(set_key, value) > distance(new_key, value)):
                    del closest_mapping[set_key]
                    del closest_mapping_swapped[value]
                    closest_mapping[new_key] = value
                    closest_mapping_swapped[value] = new_key
                    limbo.pop()
                    limbo.append((set_key, []))
                else:
                    limbo[0][1].append(value)

        closest_mapping[TIF] = closest_value
        closest_mapping_swapped[closest_value] = TIF

    new_df = pd.DataFrame([(k,v, distance(k,v), 1-distance(k,v)/len(v)) for k,v in closest_mapping.items()], columns = (["1", "2", "distance", "accuracy"]))

    return new_df, closest_mapping_swapped[None]

In [24]:
geojson_file = 'Boundaries - Tax Increment Financing Districts.geojson'
json_tifs = extract_names_from_geojson(geojson_file)

In [25]:
matches, nones = find_closest_match(json_tifs, tif_data["tif_name"].unique(), False)

In [26]:
matches

,1,2,distance,accuracy
0,116th/Avenue O,116th/Avenue O,0,1.000000
1,Bryn Mawr/Broadway,Bryn Mawr/Broadway,0,1.000000
2,51st and Lake Park,51st/Lake Park,5,0.642857
3,Lakefront,Lakefront,0,1.000000
4,Madden/Wells,Madden/Wells,0,1.000000
5,Ohio/Wabash,Ohio/Wabash,0,1.000000
6,Stony Island Avenue Commercial and Burnside In...,Commercial Avenue,50,-1.941176
7,Homan-Arthington,Homan/Arthington,1,0.937500
8,Wilson Yard,Wilson Yard,0,1.000000
9,Montrose/Clarendon,Montrose/Clarendon,0,1.000000


In [27]:
# matches.loc[matches['1'] == "105th/Vincennes", '2'] = "105th Street and Vincennes Avenue"
# matches.loc[matches['1'] == "111th Street/Kedzie Avenue Business District", '2'] = "111th/Kedzie"
# matches.loc[matches['1'] == "Stony Island Avenue Commercial and Burnside Industrial Corridors", '2'] = "Stony Island Commercial/Burnside Industrial"
# matches.loc[matches['1'] == "43rd/Cottage Grove", '2'] = "43rd Street/Cottage Grove Avenue"
# matches.loc[matches['1'] == "47th/King Drive", '2'] = "47th and King Drive"
# matches.loc[matches['1'] == "Commercial Avenue", '2'] = "Commercial Avenue"
# matches.loc[matches['1'] == "Kinzie Industrial Corridor", '2'] = "Kinzie Industrial Conservation Area"
# matches.loc[matches['1'] == "Woodlawn", '2'] = "Woodlawn"
# matches.loc[matches['1'] == "Red Purple Transit", '2'] = "Red and Purple Modernization Phase One Project"

matches.loc[matches['1'] == "Stony Island Avenue Commercial and Burnside Industrial Corridors", '2'] = "Stony Island Avenue Commercial and Burnside Industrial Corridors"
matches.loc[matches['1'] == "Kinzie Industrial Conservation Area", '2'] = "Kinzie Industrial Corridor"
matches.loc[matches['1'] == "Woodlawn", '2'] = "Woodlawn"
matches.loc[matches['1'] == "111th Street/Kedzie Avenue Business District", '2'] = "111th/Kedzie"

In [28]:
conversions = {row["2"]:row["1"] for index, row in matches.iterrows()}
tif_data["tif_name"] = tif_data["tif_name"].replace(conversions)
tif_data['tif_name'].unique()

array(['105th Street and Vincennes Avenue', '107th/Halsted',
       '111th Street/Kedzie Avenue Business District', '116th/Avenue O',
       '119th and Halsted', '119th Street/I-57', '126th/Torrence',
       '134th/Avenue K', '24th/Michigan', '26th & King Drive',
       '35th/Halsted', '35th/State', '35th/Wallace', '41st/King',
       '43rd Street/Cottage Grove Avenue', '45th/Western', '47th/Ashland',
       '47th/Halsted', '47th and King Drive', '47th/State',
       '49th/St. Lawrence', '51st/Archer', '51st and Lake Park',
       '53rd Street', '60th/Western', '63rd/Ashland', '63rd/Pulaski',
       '67th/Cicero', '67th and Wentworth', '69th/Ashland',
       '71st & Stony Island', '73rd/University', '79th Street Corridor',
       '79th and Cicero', '79th Street/Southwest Highway',
       '79th/Vincennes', '83rd/Stewart', '87th/Cottage Grove',
       '95th/Stony Island', '95th & Western', 'Addison North',
       'Addison South', 'Archer Courts', 'Archer/Central',
       'Archer/Western'

In [29]:
# Cumulative in and out
tif_data['cumulative_transfers_in'] = 0
tif_data['cumulative_transfers_out'] = 0
tif_data['cumulative_property_tax_extraction'] = 0
for index, row in tif_data.iterrows():
    local_data = tif_data[(tif_data['tif_name'] == row['tif_name']) & (tif_data['property_tax_extraction'] != 0) & (tif_data['tif_year'] <= row['tif_year'])]
    tif_data.loc[index, 'cumulative_transfers_in'] = local_data['transfers_in'].sum()
    tif_data.loc[index, 'cumulative_transfers_out'] = local_data['transfers_out'].sum()
    tif_data.loc[index, 'cumulative_property_tax_extraction'] = local_data['property_tax_extraction'].sum()

In [30]:
tif_data.head(20)

,tif_name,tif_year,property_tax_extraction,transfers_in,transfers_out,cumulative_transfers_in,cumulative_transfers_out,cumulative_property_tax_extraction
0,105th Street and Vincennes Avenue,2014,133001,353381,0,353381,0,133001
1,105th Street and Vincennes Avenue,2015,648627,353381,0,706762,0,781628
2,105th Street and Vincennes Avenue,2016,768234,0,0,706762,0,1549862
3,105th Street and Vincennes Avenue,2017,1190748,0,0,706762,0,2740610
4,105th Street and Vincennes Avenue,2018,1038646,706761,0,1413523,0,3779256
5,105th Street and Vincennes Avenue,2019,778591,0,0,1413523,0,4557847
6,105th Street and Vincennes Avenue,2020,759998,0,0,1413523,0,5317845
7,105th Street and Vincennes Avenue,2021,1311739,0,0,1413523,0,6629584
8,105th Street and Vincennes Avenue,2022,1383445,0,0,1413523,0,8013029
9,107th/Halsted,2014,0,0,0,0,0,0


In [31]:
tif_data.to_csv("PhillipYates/Master_TIF_formatted.csv")